## Import relevant packages

In [50]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Data

In [51]:
mnist_dataset, mnist_info = tfds.load(name = 'mnist', with_info = True, as_supervised = True)

mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = 0.1 * mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)


def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)

test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

# Model

### Outline the model 

In [76]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape = (28,28,1)),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'tanh'),
    tf.keras.layers.Dense(output_size, activation = 'softmax')
])

## Choose the optimizer and the loss function 

In [79]:
custom_optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer = custom_optimizer, loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

## Training

In [80]:
NUM_EPOCHS = 5

model.fit(train_data, epochs = NUM_EPOCHS, validation_data = (validation_inputs, validation_targets), validation_steps = 1, verbose = 2)

Epoch 1/5
540/540 - 8s - loss: 0.0236 - accuracy: 0.9936 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00
Epoch 2/5
540/540 - 7s - loss: 0.0192 - accuracy: 0.9952 - val_loss: 0.0309 - val_accuracy: 0.9912
Epoch 3/5
540/540 - 7s - loss: 0.0165 - accuracy: 0.9962 - val_loss: 0.0282 - val_accuracy: 0.9925
Epoch 4/5
540/540 - 7s - loss: 0.0152 - accuracy: 0.9969 - val_loss: 0.0257 - val_accuracy: 0.9928
Epoch 5/5
540/540 - 7s - loss: 0.0136 - accuracy: 0.9973 - val_loss: 0.0237 - val_accuracy: 0.9938


## Test the model

In [81]:
test_loss, test_accuracy = model.evaluate(test_data)

10/10 [==============================] - 2s 177ms/step - loss: 0.0625 - accuracy: 0.9799


In [82]:
print('Test loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

Test loss: 0.06. Test accuracy: 97.99%
